In [1]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import import_ipynb
import tqdm
from torchvision import datasets
from torch.autograd import Variable
from model.resnet_l import ResNet50

import json
import os

importing Jupyter notebook from /home/u2022171199/jupyterlab/KDtest/model/resnet_l.ipynb


In [2]:
def get_train_val(train_dir, val_dir, batch_size):
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 3.0)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])

    dataset_train = datasets.ImageFolder(train_dir, transform=transform)
    dataset_test = datasets.ImageFolder(val_dir, transform=transform_test)
    with open('class.txt', 'w', encoding='utf-8') as file:
        file.write(str(dataset_train.class_to_idx))
    with open('class.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(dataset_train.class_to_idx))
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size = batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size = batch_size, shuffle=False)
    return train_loader,test_loader

In [3]:
def train(model, device, train_loader, test_loader, epoch, learing_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    cosine_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20, eta_min=1e-9)
    val_acc_list= {}
    print(device)
    for epoch in range(1, epoch + 1):
        model.train()
        sum_loss = 0
        total_num = len(train_loader.dataset)
        print(total_num, len(train_loader.dataset))
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data).to(device), Variable(target).to(device)
            _, _, _, _, _, out = model(data)
            loss = criterion(out, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print_loss = loss.data.item()
            sum_loss += print_loss
            if (batch_idx + 1) % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
        ave_loss = sum_loss/len(train_loader)
        print('epoch:{},loss:{}'.format(epoch, ave_loss))
        cosine_schedule.step()
        acc = val(model, device, test_loader, criterion)
        val_acc_list[epoch] = acc
        with open('result.json', 'w', encoding='utf-8') as file:
            file.write(json.dumps(val_acc_list))
    torch.save(model, 'teacherNet/model_final.pth')

In [4]:
best_acc = 0
def val(model, device, test_loader,criterion):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            _, _, _, _, _, out = model(data)
            loss = criterion(out, target)
            _, pred = torch.max(out.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        if acc > best_acc:
            torch.save(model, file_dir + '/' + 'best.pth')
            best_acc = acc
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
        return acc

In [5]:
if __name__ == '__main__':
    file_dir = 'teacherNet'
    if not os.path.exists(file_dir):
        os.makedirs(file_dir)


In [6]:
    learning_rate = 1e-4
    batch_size = 64
    epoch = 100
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
    train_dir = './data/train'
    val_dir = './data/val'
    train_loader, test_loader = get_train_val(train_dir, val_dir, batch_size)

In [8]:
    model = ResNet50()
    model.fc = nn.Linear(model.fc.in_features,12)#对全连接输出分类数做修改
    model.to(device)
    train(model,device,train_loader,test_loader,epoch,learning_rate)

cuda
3961 3961
Train Epoch: 1 [640/3961 (16%)]	Loss: 2.380528
Train Epoch: 1 [1280/3961 (32%)]	Loss: 1.996207
Train Epoch: 1 [1920/3961 (48%)]	Loss: 2.157241
Train Epoch: 1 [2560/3961 (65%)]	Loss: 1.903153
Train Epoch: 1 [3200/3961 (81%)]	Loss: 2.012834
Train Epoch: 1 [3840/3961 (97%)]	Loss: 2.030647
epoch:1,loss:2.151450620543572

Val set: Average loss: 3.3349, Accuracy: 124/1698 (7%)

3961 3961
Train Epoch: 2 [640/3961 (16%)]	Loss: 1.910794
Train Epoch: 2 [1280/3961 (32%)]	Loss: 1.757392
Train Epoch: 2 [1920/3961 (48%)]	Loss: 1.619478
Train Epoch: 2 [2560/3961 (65%)]	Loss: 1.881051
Train Epoch: 2 [3200/3961 (81%)]	Loss: 1.814469
Train Epoch: 2 [3840/3961 (97%)]	Loss: 1.547385
epoch:2,loss:1.750547347530242

Val set: Average loss: 2.2606, Accuracy: 506/1698 (30%)

3961 3961
Train Epoch: 3 [640/3961 (16%)]	Loss: 1.543756
Train Epoch: 3 [1280/3961 (32%)]	Loss: 1.459021
Train Epoch: 3 [1920/3961 (48%)]	Loss: 1.335726
Train Epoch: 3 [2560/3961 (65%)]	Loss: 1.301329
Train Epoch: 3 [3200/39